# Fluid‑HD Simulation & Binding

In this notebook we:
1. Simulate a fluid “nightclub” of agents with overcrowding, goal‑drift, and jitter.
2. Build high‑dimensional state embeddings.
3. Detect “hop” events when agents change groups.
4. Bind each (i→j) hop into a Hyperdimensional memory vector.
5. Save the resulting `hop_log` and `M_memory` for later analysis.

## 1.1. Setup & Imports

In [ ]:
# Parameters (tweak these!)
N              = 100           # number of agents
D              = 4096          # HD dimension
tau_intra      = 0.85          # intra‑group threshold
tau_inter      = 0.35          # inter‑group threshold
T_steps        = 5000          # total simulation steps
normalize_every= 500           # memory normalization interval
seed           = 42

# Path setup
import sys, os
sys.path.append(os.path.abspath('../src'))

## 1.2. Set Hyperparameters

Here, we define the key parameters for our simulation run. These can be tuned to explore different dynamics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from simulation import AgentSimulator
from affinity   import compute_hd_embeddings, cosine_affinity
from grouping   import build_adjacency, extract_groups
from binding    import HDBinder
from memory     import MemoryStore
from utils      import set_seed

# reproducibility
set_seed(seed)

# instantiate modules
sim    = AgentSimulator(N=N, seed=seed)
binder = HDBinder(N=N, D=D, seed=seed)
memory = MemoryStore(D=D)

## 1.3. Instantiate Core Components

In [ ]:
hop_log     = []            # list of (t, i, j)
prev_groups = np.zeros(N, int)

for t in range(T_steps):
    # 1) advance simulator → states shape (N,4)
    states = sim.step()

    # 2) project to HD state embeddings H: shape (N,D)
    H = compute_hd_embeddings(states, binder.proj_W)

    # 3) build affinity graph S → adjacency A
    S = cosine_affinity(H)
    A = build_adjacency(S, prev_groups, tau_intra, tau_inter)

    # 4) extract current groups
    curr_groups = extract_groups(A)
    
    # 5) detect hops and bind
    for i in range(N):
        if curr_groups[i] != prev_groups[i]:
            members = np.where(curr_groups == curr_groups[i])[0]
            for j in members:
                hop_log.append((t, i, j))
                binder.record_hop(memory, i, j)

    prev_groups = curr_groups.copy()

    # 6) optional memory normalization
    if (t+1) % normalize_every == 0:
        memory.normalize()

## 1.4. Run Simulation Loop

In [ ]:
# inspect first few hops
print('First 10 hops:', hop_log[:10])

# memory stats
M = memory.M
print('Memory distribution:', np.min(M), np.max(M), np.mean(M))
plt.hist(M, bins=21)
plt.title('Memory Value Distribution')
plt.show()

### Quick Visualization: Agent Trajectories

Let's plot the paths of the first few agents to see what the simulation looks like.

In [ ]:
plt.figure(figsize=(8, 8))
for i in range(min(N, 10)):
    plt.plot(state_history[:, i, 0], state_history[:, i, 1], alpha=0.7)
plt.title('Trajectories of First 10 Agents')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.grid(True)
plt.show()

## 1.5. Save Results

We now save the key artifacts to disk. These will be loaded by the second notebook for evaluation and visualization.

In [ ]:
# collect first 200 steps for animation
sim.reset()
prev = np.zeros(N, int)
pos_list, grp_list = [], []
for t in range(200):
    pos = sim.agent_positions()
    states = sim.step()
    H = compute_hd_embeddings(states, binder.proj_W)
    A = build_adjacency(cosine_affinity(H), prev, tau_intra, tau_inter)
    grp = extract_groups(A)
    pos_list.append(pos.copy())
    grp_list.append(grp.copy())
    prev = grp

# animate inline
import matplotlib.animation as animation
fig, ax = plt.subplots()
scat = ax.scatter([], [], c=[], cmap='tab20', s=50)
def update(frame):
    pts = pos_list[frame]
    colors = grp_list[frame]
    scat.set_offsets(pts)
    scat.set_array(colors)
    ax.set_title(f'Step {frame}')
    return scat,
ani = animation.FuncAnimation(fig, update, frames=200, blit=True)
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
import pickle
import numpy as np

os.makedirs('../results', exist_ok=True)
with open('../results/hop_log.pkl','wb') as f:
    pickle.dump(hop_log, f)
np.savez('../results/M_memory.npz', M=memory.M)
